In [1]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

numInitSubstrate = 20
numInitEnzyme = 10
numTraces = 1000

numTimeSteps = 100
simulationTime = 2.5

dt = simulationTime/numTimeSteps

q1 = numInitEnzyme                            # there are no complex molecules at beginning, therefore q1 = X(2) = 10

c1 = 1.2                                       # some reaction rates
c2 = 0.3                                      
c3 = 2.9                                      

# Define the state-transition vectors and propensity functions
v1 = [-1,-1];
v2 = [1,1];
v3 = [0,1];

a1 = lambda Substrate, Enzyme: Substrate*Enzyme*c1
a2 = lambda Enzyme: (q1-Enzyme)*c2
a3 = lambda Enzyme: (q1-Enzyme)*c3

# Instead of Calculating directly on P, we will sample a given number of
# traces. For each trace we store state and time.

TraceTimes = [0 for i in range(numTraces)]
Trace_Substrate_State = [0 for i in range(numTraces)]
Trace_Enzyme_State = [0 for i in range(numTraces)]

# Initialize the state - each simulation starts with the same condition
Trace_Substrate_State[:] = [numInitSubstrate for i in range(numTraces)]
Trace_Enzyme_State[:] = [numInitEnzyme for i in range(numTraces)]



def calcPFromTraces(tSteps, Trace_Substrate_State, Trace_Enzyme_State ,numInitSubstrate,numInitEnzyme):

    # Initialize as 0
    P = pd.DataFrame([[0 for i in range(numInitEnzyme+1)] for i in range(numInitSubstrate+1)])    
      # There are at most 20 Substrates and 10 Enzymes 

    # loop over all traces to accumulate the states of each specific one
    for T in range(len(Trace_Substrate_State)):
        P.iloc[Trace_Substrate_State[T],Trace_Enzyme_State[T]] = P.iloc[Trace_Substrate_State[T],Trace_Enzyme_State[T]] + 1
    
    # normalize and print heatmap
    P = P/len(Trace_Substrate_State)
    sns.heatmap(P, annot=True)
    plt.xlabel('# Enzymes')
    plt.ylabel('# Substrates') 
    plt.savefig('Tau_leaping_traces/Tau_leaping_trace' + str(tSteps)+ '.png')
    plt.close()
    return P



for tSteps in range(numTimeSteps): # loop over all time steps
    for T in range(numTraces): # loop over all traces
        
        Enzyme, Substrate  = Trace_Enzyme_State[T], Trace_Substrate_State[T] # copy current state into local state variables
       
        # calculate propensities
        a1X = a1(Substrate, Enzyme)
        a2X = a2(Enzyme)
        a3X = a3(Enzyme)
            
        # calculate the number of each reactions happening
        numReactions1 = np.random.poisson(dt*a1X)
        numReactions2 = np.random.poisson(dt*a2X)
        numReactions3 = np.random.poisson(dt*a3X)
        
        # check if all educts needed are available
        if (numReactions1 > Substrate or numReactions1 > Enzyme):
            numReactions1 = min([Substrate, Enzyme, numReactions1])            
            
        
        if ((numReactions2 + numReactions3) > (q1-Enzyme)):
            numReactions2 = round((q1-Enzyme)*numReactions2/(numReactions2 + numReactions3))
            numReactions3 = (q1-Enzyme)-numReactions2
            
        
        # Update state
        Substrate = Substrate + numReactions1*v1[0] + numReactions2*v2[0] + numReactions3*v3[0]
        Enzyme = Enzyme + numReactions1*v1[1] + numReactions2*v2[1] + numReactions3*v3[1]
        
        # Copy local state variable to TraceStates vector
        Trace_Substrate_State[T] = Substrate
        Trace_Enzyme_State[T] = Enzyme
         
    # Calculte P from all Traces
    P = calcPFromTraces(tSteps, Trace_Substrate_State, Trace_Enzyme_State, numInitSubstrate, numInitEnzyme)                          

    

In [2]:
import imageio

# Build GIF
with imageio.get_writer('Tau_leaping_gif.gif', mode='I') as writer:
    for filename in range(numTimeSteps):
        image = imageio.imread('Tau_leaping_traces/Tau_leaping_trace' + str(filename)+ '.png')
        writer.append_data(image)